In [14]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import objectives
from keras.preprocessing import sequence

def create_lstm_vae(input_dim, timesteps, batch_size, intermediate_dim, latent_dim,epsilon_std=1.):

    x = Input(shape=(timesteps, input_dim))

    # LSTM encoding
    h = LSTM(intermediate_dim)(x)

    # VAE Z layer
    z_mean = Dense(latent_dim)(h)
    z_log_sigma = Dense(latent_dim)(h)
    
    def sampling(args):
        z_mean, z_log_sigma = args
        epsilon = K.random_normal(shape=(batch_size, latent_dim),
                                  mean=0., stddev=epsilon_std)
        return z_mean + z_log_sigma * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    # so you could write `Lambda(sampling)([z_mean, z_log_sigma])`
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_sigma])
    
    # decoded LSTM layer
    decoder_h = LSTM(intermediate_dim, return_sequences=True)
    decoder_mean = LSTM(input_dim, return_sequences=True)

    h_decoded = RepeatVector(timesteps)(z)
    h_decoded = decoder_h(h_decoded)

    # decoded layer
    x_decoded_mean = decoder_mean(h_decoded)
    
    # end-to-end autoencoder
    vae = Model(x, x_decoded_mean)

    # encoder, from inputs to latent space
    encoder = Model(x, z_mean)

    # generator, from latent space to reconstructed inputs
    decoder_input = Input(shape=(latent_dim,))

    _h_decoded = RepeatVector(timesteps)(decoder_input)
    _h_decoded = decoder_h(_h_decoded)

    _x_decoded_mean = decoder_mean(_h_decoded)
    generator = Model(decoder_input, _x_decoded_mean)
    
    def vae_loss(x, x_decoded_mean):
        xent_loss = objectives.mse(x, x_decoded_mean)
        kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
        loss = xent_loss + kl_loss
        return loss

    vae.compile(optimizer='adam', loss=vae_loss)
    
    return vae, encoder, generator

In [15]:
import glob
import numpy as np
from sklearn.model_selection import train_test_split
dir = './sequence/*'
# file read
all_data = []
sequence_length = []
for file in sorted(glob.glob(dir)):
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(',')
        datasets.append([int(u), int(v), float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [16]:
# timesteps 맞춰주기 위해서 padding 추가
max_sequence_length = max(sequence_length)
all_data = sequence.pad_sequences(all_data, padding='post', maxlen = max_sequence_length)

In [17]:
x_train, x_test = train_test_split(all_data, test_size = 0.2)
x_test, x_val = train_test_split(x_test, test_size = 0.66)

In [18]:
vae, enc, gen = create_lstm_vae(3, 
        timesteps=max_sequence_length, 
        batch_size=1, 
        intermediate_dim=32,
        latent_dim=128,
        epsilon_std=1.)

In [19]:
vae.fit(x_train, x_train, epochs=1000, batch_size=25,  validation_data=(x_val, x_val))

Train on 540 samples, validate on 90 samples
Epoch 1/1000
540/540 [==============================] - 2s 4ms/step - loss: 13.5286 - val_loss: 12.6100
Epoch 2/1000
540/540 [==============================] - 1s 1ms/step - loss: 11.9615 - val_loss: 11.3449
Epoch 3/1000
540/540 [==============================] - 1s 1ms/step - loss: 11.0730 - val_loss: 10.9387
Epoch 4/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.8366 - val_loss: 10.8613
Epoch 5/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.7962 - val_loss: 10.8314
Epoch 6/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.7748 - val_loss: 10.8157
Epoch 7/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.7609 - val_loss: 10.8050
Epoch 8/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.7493 - val_loss: 10.7953
Epoch 9/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.7410 - val_loss: 10.7888
Epoch 10/1000
540/5

540/540 [==============================] - 1s 1ms/step - loss: 10.6785 - val_loss: 10.7289
Epoch 79/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6785 - val_loss: 10.7290
Epoch 80/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6784 - val_loss: 10.7289
Epoch 81/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6784 - val_loss: 10.7291
Epoch 82/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6787 - val_loss: 10.7293
Epoch 83/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6787 - val_loss: 10.7296
Epoch 84/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6785 - val_loss: 10.7288
Epoch 85/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6781 - val_loss: 10.7284
Epoch 86/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6780 - val_loss: 10.7284
Epoch 87/1000
540/540 [==============================] - 1s 1ms/step 

540/540 [==============================] - 1s 1ms/step - loss: 10.6757 - val_loss: 10.7262
Epoch 156/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6756 - val_loss: 10.7262
Epoch 157/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6760 - val_loss: 10.7260
Epoch 158/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6761 - val_loss: 10.7266
Epoch 159/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6758 - val_loss: 10.7259
Epoch 160/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6756 - val_loss: 10.7258
Epoch 161/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6753 - val_loss: 10.7257
Epoch 162/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6753 - val_loss: 10.7257
Epoch 163/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6753 - val_loss: 10.7256
Epoch 164/1000
540/540 [==============================] - 1s 

540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7249
Epoch 233/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7248
Epoch 234/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7248
Epoch 235/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7248
Epoch 236/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7249
Epoch 237/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7248
Epoch 238/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7248
Epoch 239/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6745 - val_loss: 10.7249
Epoch 240/1000
540/540 [==============================] - 1s 1ms/step - loss: 10.6744 - val_loss: 10.7248
Epoch 241/1000
540/540 [==============================] - 1s 

KeyboardInterrupt: 